<a href="https://colab.research.google.com/github/crescendonow/thai_geoparsing/blob/main/geoparsing_colab/1_3BERT_polynomial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Utilities lib for use
import requests
import numpy as np
import pickle
import tqdm
import requests
import torch
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification,\
TrainingArguments, Trainer, DataCollatorForTokenClassification

In [ ]:
#load train, validate and test token by attacut and BERT format
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\1.Tokennization\train_bert.data', 'rb') as token1:
    train_data = pickle.load(token1)
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\1.Tokennization\eval_bert.data', 'rb') as token2:
    eval_data = pickle.load(token2)
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\1.Tokennization\datasets_test.data', 'rb') as token3:
    test_data = pickle.load(token3)    

In [ ]:
#load dictionary index and processing data label_list
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\5.dict\ner_ix.dict', 'rb') as dict_ner:
    ner_ix = pickle.load(dict_ner)
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\6.Processing_variables\ner_list.data', 'rb') as list_ner:
    label_list = pickle.load(list_ner)

In [ ]:
#prepared index to BOI tags, convert index to ner
ix_to_ner = dict((v,k) for k,v in ner_ix.items()) 

In [ ]:
#list of model for use
task = "ner"
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased'] 
user_name = 'airesearch'
model_name = "wangchanberta-base-att-spm-uncased" 
batch_size = 16

In [ ]:
#define tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'{user_name}/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

loading configuration file https://huggingface.co/airesearch/wangchanberta-base-att-spm-uncased/resolve/main/config.json from cache at C:\Users\sveng/.cache\huggingface\transformers\616a9e2dfc52e9d019b75d219ed800a27158ed299bd4fad91363110fe93dfce1.27c4f6581fbedf3d12e9fae96d4fbb8bc3064cd88ae545414e7cffc7c5bbc52f
Model config CamembertConfig {
  "_name_or_path": "airesearch/wangchanberta-base-att-spm-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "camembert",
  "num_attention_head": 12,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_versio

In [ ]:
#list of model for use
task = "ner"
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased'] 
user_name = 'airesearch'
model_name = "wangchanberta-base-att-spm-uncased" 
batch_size = 16

In [ ]:
#create example for use tokenize and input ids
example = train_data[1617]
tokenized_input = tokenizer(example['tokens'], is_split_into_words=True)
tokenized_input
print(len(tokenized_input['input_ids']), len(example['ner_tags']))#create example for use tokenize and input ids

10 5


In [ ]:
#from upper cell rcognize length of word_ids and tags not equal so we need to algn new label 
aligned_labels = [-100 if i is None else example[f'{task}_tags'][i] for i in tokenized_input.word_ids()]
print(len(tokenized_input['input_ids']), len(aligned_labels))
aligned_labels
tokenized_input

10 10


{'input_ids': [5, 10, 2274, 874, 775, 10, 17, 6100, 627, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#create function for tokenize and reformat
def tokenize_and_align_labels(data):
    tokenized_datasets = []
    for rec in data:
        tokenized_input = tokenizer(rec['tokens'], is_split_into_words=True)
        aligned_labels = [-100 if i is None else rec[f'{task}_tags'][i] for i in tokenized_input.word_ids()]
        rec['input_ids'] = tokenized_input['input_ids']
        rec['attention_mask'] = tokenized_input['attention_mask']
        rec['labels'] = aligned_labels
        tokenized_datasets.append(rec)
    return tokenized_datasets

In [ ]:
train_bert = tokenize_and_align_labels(train_data)
eval_bert = tokenize_and_align_labels(eval_data)
test_bert = tokenize_and_align_labels(test_data)

In [ ]:
#prepare fine tuning model
model = AutoModelForTokenClassification.from_pretrained(f'{user_name}/{model_name}', num_labels=len(label_list))

#load model from state 20 epoch
path_chkpoint = r'C:\Users\sveng\geoparsing_proj\model_bert\Model1-v120\checkpoint-22000\pytorch_model.bin'
model.load_state_dict(torch.load(path_chkpoint))

loading configuration file https://huggingface.co/airesearch/wangchanberta-base-att-spm-uncased/resolve/main/config.json from cache at C:\Users\sveng/.cache\huggingface\transformers\616a9e2dfc52e9d019b75d219ed800a27158ed299bd4fad91363110fe93dfce1.27c4f6581fbedf3d12e9fae96d4fbb8bc3064cd88ae545414e7cffc7c5bbc52f
Model config CamembertConfig {
  "_name_or_path": "airesearch/wangchanberta-base-att-spm-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "L

<All keys matched successfully>

In [ ]:
#training arguments (parameters)
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy="epoch",

    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=10,

    seed=9,
    warmup_ratio=0.05,
    lr_scheduler_type="polynomial",

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#function for convert dataset to batch tensors
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
#computation metrics , we use seqeval for sequence tagging task
metric = load_metric("seqeval")

In [ ]:
#for seqeval not allowed format '-' so replace it by '_', pong tai i here tam goo sia wela eak, sud!!
labels = [label_list[i].replace('_', '-') for i in example[f'{task}_tags']]
metric.compute(predictions=[labels], references=[labels])

c:\Users\sveng\anaconda3\envs\geo_bert\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sveng\anaconda3\envs\geo_bert\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sveng\anaconda3\envs\geo_bert\lib\site-packages\numpy\lib\function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\Users\sveng\anaconda3\envs\geo_bert\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

In [ ]:
#create function for computation metric
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
#fine tuning part by function Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=train_bert,
    eval_dataset=eval_bert,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
%%time
#call method train
trainer.train()


c:\Users\sveng\anaconda3\envs\geo_bert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17955
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 5610


  0%|          | 0/5610 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags, pos_tags. If id, tokens, ner_tags, pos_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to test-ner\checkpoint-500
Configuration saved in test-ner\checkpoint-500\config.json


{'loss': 0.0062, 'learning_rate': 1.918219178082192e-05, 'epoch': 0.89}


Model weights saved in test-ner\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-ner\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-ner\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4489
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags, pos_tags. If id, tokens, ner_tags, pos_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


...

Model weights saved in test-ner\checkpoint-5500\pytorch_model.bin
tokenizer config file saved in test-ner\checkpoint-5500\tokenizer_config.json
Special tokens file saved in test-ner\checkpoint-5500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4489
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags, pos_tags. If id, tokens, ner_tags, pos_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/281 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.13687586784362793, 'eval_precision': 0.6268372346216657, 'eval_recall': 0.6837885985748219, 'eval_f1': 0.6540755467196819, 'eval_accuracy': 0.9798924358084663, 'eval_runtime': 230.7492, 'eval_samples_per_second': 19.454, 'eval_steps_per_second': 1.218, 'epoch': 10.0}
{'train_runtime': 30847.5973, 'train_samples_per_second': 5.821, 'train_steps_per_second': 0.182, 'train_loss': 0.005496271132572874, 'epoch': 10.0}
CPU times: total: 2d 20h 24min 15s
Wall time: 8h 34min 7s


TrainOutput(global_step=5610, training_loss=0.005496271132572874, metrics={'train_runtime': 30847.5973, 'train_samples_per_second': 5.821, 'train_steps_per_second': 0.182, 'train_loss': 0.005496271132572874, 'epoch': 10.0})

In [ ]:
predictions, labels, _ = trainer.predict(test_bert)
predictions = np.argmax(predictions, axis=2)

***** Running Prediction *****
  Num examples = 22444
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags, pos_tags. If id, tokens, ner_tags, pos_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/1403 [00:00<?, ?it/s]

In [ ]:
true_predictions = [
    [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [ ]:
results = metric.compute(predictions=true_predictions, references=true_labels)
#print(results)

In [ ]:
import pandas as pd
#reformat classification report
def show_3digits(evalue):
    val = round(evalue, 4)
    return val

def eliminate_decimal(evalue):
    val = int(evalue)
    return val

classification_report = pd.DataFrame.from_dict(results).transpose()

classification_report['precision'] = classification_report['precision'].apply(show_3digits)
classification_report['recall'] = classification_report['recall'].apply(show_3digits)
classification_report['f1'] = classification_report['f1'].apply(show_3digits)
classification_report['number'] = classification_report['number'].apply(eliminate_decimal)

In [ ]:
print(classification_report)

                   precision  recall      f1  number
ACP                   0.9254  0.9171  0.9212     784
ADMIN                 0.9223  0.9342  0.9282    4332
BSN                   0.8293  0.8987  0.8626     454
DEP                   0.9303  0.9501  0.9401    1882
FPLACE                0.8832  0.9249  0.9035    1185
GOV                   0.9032  0.8725  0.8876     353
HP                    0.8918  0.9330  0.9120     433
MKT                   0.9410  0.9377  0.9393     289
MON                   0.8421  0.7442  0.7901      43
NAT                   0.8170  0.9015  0.8571     203
OTHER                 0.9333  0.8750  0.9032      32
RCT                   0.9099  0.9643  0.9363     785
RES                   0.8420  0.9093  0.8743     551
ROAD                  0.9072  0.9269  0.9170    2490
RP                    0.9568  0.9674  0.9621    1075
RT                    0.8890  0.9306  0.9093    2220
STORE                 0.8192  0.8237  0.8214     363
TRAN                  0.8710  0.9144  0.8922  

In [ ]:
#line notify to me if complete
token_jobnotify = 'M1qygx1uFYuGtgo6qoKPboB4sAKmAhGuwXdL7q54TvP'
msg = f""" run process เสร็จละนะจ๊ะที่รัก เข้ามาดูเค้าหน่อยน้า 
           f1 = {classification_report.loc['overall_f1']['f1']} """
url = 'https://notify-api.line.me/api/notify'
headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token_jobnotify}
r = requests.post(url, headers=headers, data = {'message':msg})
print (r.text)

{"status":200,"message":"ok"}


In [ ]:
#Save model by this path
folder_name = 'Model1-v3'
save_path = r"C:\Users\sveng\geoparsing_proj\model_bert"
fullpath = save_path+'\\'+folder_name

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(fullpath)

# save last weight pretrained
model.save_pretrained(fullpath)

Configuration saved in C:\Users\sveng\geoparsing_proj\model_bert\Model1-v3\config.json
Model weights saved in C:\Users\sveng\geoparsing_proj\model_bert\Model1-v3\pytorch_model.bin
